In [1]:
#imports

import pandas as pd


/home/joelpiet/.local/lib/python3.10/site-packages/pandas/core/arrays/masked.py:62: UserWarning: Pandas requires version '1.3.4' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (


Now that we have processed the HSL bike data, the next step is to process the weather data and merge it with the HSL data into a single dataframe. 

In [2]:
weather_data = pd.DataFrame()
dataframes = []

for year in range(2016, 2022):
    csv_path = "../data/weather/" + str(year) + "-weather.csv"
    df = pd.read_csv(csv_path)
    dataframes.append(df)

weather_data = pd.concat(dataframes, ignore_index=True)
display(weather_data)


,Year,m,d,Time,Time zone,Precipitation amount (mm),Air temperature (degC),Maximum temperature (degC),Minimum temperature (degC)
0,2016,5,1,00:00,UTC,-1.0,11.1,16.4,5.1
1,2016,5,2,00:00,UTC,-1.0,12.1,17.8,5.9
2,2016,5,3,00:00,UTC,-1.0,11.9,17.7,4.6
3,2016,5,4,00:00,UTC,-1.0,14.3,20.4,5.7
4,2016,5,5,00:00,UTC,-1.0,15.5,21.5,8.9
...,...,...,...,...,...,...,...,...,...
1219,2021,10,27,00:00,UTC,7.5,7.8,9.3,6.2
1220,2021,10,28,00:00,UTC,1.8,8.2,10.6,4.3
1221,2021,10,29,00:00,UTC,-1.0,9.8,10.9,8.3
1222,2021,10,30,00:00,UTC,-1.0,8.7,9.4,7.3


Now that we have processed the weather data into a dataframe, the next step is to get rid of unnecessary columns, as well as combine the year month day columns into a single "date" column.

In [3]:
weather_data = weather_data.drop(['Time', 'Time zone'], axis=1)

weather_data['Date'] = weather_data['Year'].astype(str) + '-' + weather_data['m'].astype(str) + '-' + weather_data['d'].astype(str)
weather_data['Date'] = pd.to_datetime(weather_data['Date'], format='%Y-%m-%d').dt.date
#subtract 2016 from year 
weather_data['Year'] = weather_data['Year'] - 2016

weather_data = weather_data.rename(columns={'m': 'Month', 'd' : 'Day'})

weather_data = weather_data.iloc[1:]

weather_data['Precipitation amount (mm)'] = weather_data['Precipitation amount (mm)'].replace(-1.0, 0.0)

display(weather_data)

,Year,Month,Day,Precipitation amount (mm),Air temperature (degC),Maximum temperature (degC),Minimum temperature (degC),Date
1,0,5,2,0.0,12.1,17.8,5.9,2016-05-02
2,0,5,3,0.0,11.9,17.7,4.6,2016-05-03
3,0,5,4,0.0,14.3,20.4,5.7,2016-05-04
4,0,5,5,0.0,15.5,21.5,8.9,2016-05-05
5,0,5,6,0.0,14.8,19.7,8.1,2016-05-06
...,...,...,...,...,...,...,...,...
1219,5,10,27,7.5,7.8,9.3,6.2,2021-10-27
1220,5,10,28,1.8,8.2,10.6,4.3,2021-10-28
1221,5,10,29,0.0,9.8,10.9,8.3,2021-10-29
1222,5,10,30,0.0,8.7,9.4,7.3,2021-10-30


Now that we have both the processed HSL bike data, as well as the processed weather data, we can combine the two dataframes. 

In [4]:
bike_data = pd.read_csv('../data/HSL/processed_bike_data.csv')

bike_data['Date'] = pd.to_datetime(bike_data['Date'], format='%Y-%m-%d').dt.date
finalized_data = pd.merge(bike_data, weather_data, on='Date', how='left')

display(finalized_data)


,Station,Date,Departure Count,Return Count,Year,Month,Day,Precipitation amount (mm),Air temperature (degC),Maximum temperature (degC),Minimum temperature (degC)
0,Linnanmäki,2016-05-02,11.0,15.0,0,5,2,0.0,12.1,17.8,5.9
1,Workshop Helsinki,2016-05-02,14.0,13.0,0,5,2,0.0,12.1,17.8,5.9
2,Kampin metroasema,2016-05-02,17.0,12.0,0,5,2,0.0,12.1,17.8,5.9
3,Laivasillankatu,2016-05-02,3.0,3.0,0,5,2,0.0,12.1,17.8,5.9
4,Merisotilaantori,2016-05-02,2.0,3.0,0,5,2,0.0,12.1,17.8,5.9
...,...,...,...,...,...,...,...,...,...,...,...
317556,Koivusaari (M),2021-10-31,7.0,10.0,5,10,31,0.0,8.7,9.6,7.1
317557,Komeetankatu,2021-10-31,8.0,8.0,5,10,31,0.0,8.7,9.6,7.1
317558,Toppelundintie,2021-10-31,1.0,4.0,5,10,31,0.0,8.7,9.6,7.1
317559,Torpanranta,2021-10-31,8.0,9.0,5,10,31,0.0,8.7,9.6,7.1


Now we have our dataframe that includes data regarding the HSL bikes and weather. The last thing we want to do is to add columns to our dataframe that can be used to help our model.

In [5]:
#We add a column to indicate whether the day is a weekend or not

finalized_data['Weekend'] = finalized_data['Date'].apply(lambda x: x.strftime("%A")).apply(lambda x: 1 if x in ['Saturday', 'Sunday'] else 0)
finalized_data = finalized_data.drop(['Date'], axis=1)


In [6]:
#We will also add the average returns and departures for the last 3 years for each station to the dataframe

stations = finalized_data['Station'].unique()

for i in range(3):
    for station in stations:
        for month in range(4, 11):
            month_data = finalized_data[(finalized_data['Year'] < (i+3)) & (finalized_data['Year'] >= i) & (finalized_data['Month'] == month) & (finalized_data['Station'] == station)]
            avg_departure = month_data['Departure Count'].mean()
            avg_return = month_data['Return Count'].mean()
            finalized_data.loc[(finalized_data['Year'] == (i+3)) & (finalized_data['Month'] == month) & (finalized_data['Station'] == station), 'Last 3 Year Avg Departures'] = avg_departure
            finalized_data.loc[(finalized_data['Year'] == (i+3)) & (finalized_data['Month'] == month) & (finalized_data['Station'] == station), 'Last 3 Year Avg Returns'] = avg_return


display(finalized_data)

,Station,Departure Count,Return Count,Year,Month,Day,Precipitation amount (mm),Air temperature (degC),Maximum temperature (degC),Minimum temperature (degC),Weekend,Last 3 Year Avg Departures,Last 3 Year Avg Returns
0,Linnanmäki,11.0,15.0,0,5,2,0.0,12.1,17.8,5.9,0,NaN,NaN
1,Workshop Helsinki,14.0,13.0,0,5,2,0.0,12.1,17.8,5.9,0,NaN,NaN
2,Kampin metroasema,17.0,12.0,0,5,2,0.0,12.1,17.8,5.9,0,NaN,NaN
3,Laivasillankatu,3.0,3.0,0,5,2,0.0,12.1,17.8,5.9,0,NaN,NaN
4,Merisotilaantori,2.0,3.0,0,5,2,0.0,12.1,17.8,5.9,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
317556,Koivusaari (M),7.0,10.0,5,10,31,0.0,8.7,9.6,7.1,1,NaN,NaN
317557,Komeetankatu,8.0,8.0,5,10,31,0.0,8.7,9.6,7.1,1,13.075269,12.150538
317558,Toppelundintie,1.0,4.0,5,10,31,0.0,8.7,9.6,7.1,1,3.168675,3.481928
317559,Torpanranta,8.0,9.0,5,10,31,0.0,8.7,9.6,7.1,1,6.934066,7.208791


In [7]:
#Then we get the average for the final 3 years for each station to save and use for future predictions

averages = pd.DataFrame(columns=['Station', 'Month', 'Last 3 Year Avg Departures', 'Last 3 Year Avg Returns'])

for station in stations:
        for month in range(4, 11):
            month_data = finalized_data[(finalized_data['Year'] < 6) & (finalized_data['Year'] >= 3) & (finalized_data['Month'] == month) & (finalized_data['Station'] == station)]
            avg_departure = month_data['Departure Count'].mean()
            avg_return = month_data['Return Count'].mean()
            temp = {'Station': station, 'Month': month, 'Last 3 Year Avg Departures': avg_departure, 'Last 3 Year Avg Returns': avg_return}
            averages = pd.concat([averages, pd.DataFrame(temp, index=[0])], ignore_index=True)
            if (month == 10):
                  temp = {'Station': station, 'Month': 11, 'Last 3 Year Avg Departures': avg_departure, 'Last 3 Year Avg Returns': avg_return}
                  averages = pd.concat([averages, pd.DataFrame(temp, index=[0])], ignore_index=True)

/tmp/ipykernel_10603/1450636709.py:11: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  averages = pd.concat([averages, pd.DataFrame(temp, index=[0])], ignore_index=True)


We will need to fill those NaN's in order to use the data in our model

In [10]:
#We will get the average of all the other averages for each month and fill the NaN's with that value

for i in range(3):
    for month in range(4, 11):
        avgdep = finalized_data[(finalized_data['Year'] == (i+3)) & (finalized_data['Month'] == month)]['Last 3 Year Avg Departures'].mean()
        avgret = finalized_data[(finalized_data['Year'] == (i+3)) & (finalized_data['Month'] == month)]['Last 3 Year Avg Returns'].mean()
        
        finalized_data.loc[(finalized_data['Year'] == (i+3)) & (finalized_data['Month'] == month), 'Last 3 Year Avg Departures'] = finalized_data.loc[(finalized_data['Year'] == (i+3)) & (finalized_data['Month'] == month), 'Last 3 Year Avg Departures'].fillna(avgdep)
        finalized_data.loc[(finalized_data['Year'] == (i+3)) & (finalized_data['Month'] == month), 'Last 3 Year Avg Returns'] = finalized_data.loc[(finalized_data['Year'] == (i+3)) & (finalized_data['Month'] == month), 'Last 3 Year Avg Returns'].fillna(avgret)


for month in range(4, 12):
    avgdep = averages[averages['Month'] == month]['Last 3 Year Avg Departures'].mean()
    avgret = averages[averages['Month'] == month]['Last 3 Year Avg Returns'].mean()
    
    averages.loc[averages['Month'] == month, 'Last 3 Year Avg Departures'] = averages.loc[averages['Month'] == month, 'Last 3 Year Avg Departures'].fillna(avgdep)
    averages.loc[averages['Month'] == month, 'Last 3 Year Avg Returns'] = averages.loc[averages['Month'] == month, 'Last 3 Year Avg Returns'].fillna(avgret)


display(finalized_data)

,Station,Departure Count,Return Count,Year,Month,Day,Precipitation amount (mm),Air temperature (degC),Maximum temperature (degC),Minimum temperature (degC),Weekend,Last 3 Year Avg Departures,Last 3 Year Avg Returns
0,Linnanmäki,11.0,15.0,0,5,2,0.0,12.1,17.8,5.9,0,NaN,NaN
1,Workshop Helsinki,14.0,13.0,0,5,2,0.0,12.1,17.8,5.9,0,NaN,NaN
2,Kampin metroasema,17.0,12.0,0,5,2,0.0,12.1,17.8,5.9,0,NaN,NaN
3,Laivasillankatu,3.0,3.0,0,5,2,0.0,12.1,17.8,5.9,0,NaN,NaN
4,Merisotilaantori,2.0,3.0,0,5,2,0.0,12.1,17.8,5.9,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
317556,Koivusaari (M),7.0,10.0,5,10,31,0.0,8.7,9.6,7.1,1,25.397696,25.338678
317557,Komeetankatu,8.0,8.0,5,10,31,0.0,8.7,9.6,7.1,1,13.075269,12.150538
317558,Toppelundintie,1.0,4.0,5,10,31,0.0,8.7,9.6,7.1,1,3.168675,3.481928
317559,Torpanranta,8.0,9.0,5,10,31,0.0,8.7,9.6,7.1,1,6.934066,7.208791


Then we can save it to a csv file.

In [11]:
averages.to_csv('../data/last_3_year_averages.csv', index=False)

finalized_data.to_csv('../data/finalized_data.csv', index=False)